# **CA3-Part2, LLMs Spring 2025**

- **Name:**
- **Student ID:**

---
#### Your submission should be named using the following format: `CA3-Part2_LASTNAME_STUDENTID.ipynb`.

---

##### *How to do this problem set:*

- Some questions require writing Python code and computing results, and the rest of them have written answers. For coding problems, you will have to fill out all code blocks under each `Completion` section.

- For text-based answers, you should replace the text that says `WRITE YOUR ANSWER HERE` with your actual answer, or you can look for `Report` and `Question` blocks.

- There is no penalty for using AI assistance on this homework as long as you fully disclose it in the final cell of this notebook (this includes storing any prompts that you feed to large language models). That said, anyone caught using AI assistance without proper disclosure will receive a zero on the assignment (we have several automatic tools to detect such cases). We're literally allowing you to use it with no limitations, so there is no reason to lie!

---

##### *Academic honesty*

- We will audit the Colab notebooks from a set number of students, chosen at random. The audits will check that the code you wrote actually generates the answers in your notebook. If you turn in correct answers on your notebook without code that actually generates those answers, we will consider this a serious case of cheating.

- We will also run automatic checks of Colab notebooks for plagiarism. Copying code from others is also considered a serious case of cheating.

---

If you have any further questions or concerns, contact the TAs via email or Telegram.

# RAG (50 points)

If you have any further questions or concerns, contact the TA via email (pouya.sadeghi@ut.ac.ir) or telegram.

## Install Requirements

In [1]:
!pip install -q langchain langchain_community langchain_huggingface huggingface_hub
!pip install -q sentence_transformers tiktoken lark datasets openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5

In [2]:
# To determine your system's CUDA version, run the following command:
!nvidia-smi

# Based on your CUDA version, install the appropriate FAISS-GPU package:

# For CUDA 12.x:
# !pip install -q faiss-gpu-cu12

# For CUDA 11.x:
# !pip install faiss-gpu-cu11

# If you prefer the CPU-only version of FAISS:
!pip install -q faiss-cpu

Wed Jul 16 08:49:32 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   48C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## 1. An Overview of Information Retrieval (IR) and RAG (2 points)


- **Information Retrieval (IR)**: The process of obtaining information system resources relevant to a specific information need from a collection of those resources. Each IR system consists of a collection of documents, a set of queries, and a retrieval function that ranks the documents based on their relevance to the query.
- **Retrieval-Augmented Generation (RAG)**: A model that combines the strengths of retrieval-based and generation-based approaches. It retrieves relevant documents from a large corpus and uses them to generate a response to a query. RAG is particularly useful for tasks where the answer is not explicitly present in the training data but can be inferred from related documents.
- **RAG Architecture**: The RAG architecture consists of two main components:
  - **Retriever**: This component retrieves relevant documents from a large corpus based on the input query. It can be implemented using various retrieval methods, such as BM25 or dense retrieval.
  - **Generator**: This component generates a response based on the retrieved documents and the input query. It can be implemented using transformer-based models.
  
In this computer assignment, you will implement a RAG pipeline using the LangChain framework. You will use two different retrievers: TF-IDF and dense retriever.

#### Question 1: (2 points)
1. Why do we need to use RAG?
2. What is LangChain and how does it help in building RAG pipelines?
<!-- 2. What are the advantages of RAG over generation methods? -->
<!-- 3. What is the difference between dense and sparse retrievers? -->

1.  **Why do we need to use RAG?**
    We need to use RAG primarily to address the inherent limitations of standalone Large Language Models (LLMs). LLMs are trained on a fixed dataset, meaning their knowledge is static and quickly becomes outdated concerning recent events or niche information. They also suffer from a tendency to "hallucinate," generating plausible but factually incorrect information. Furthermore, LLMs have a limited context window, preventing them from directly processing an entire large knowledge base. RAG overcomes these issues by allowing LLMs to retrieve relevant, up-to-date information from an external corpus and use it to ground their responses, thereby reducing hallucinations and providing more accurate and verifiable answers.

2.  **What is LangChain and how does it help in building RAG pipelines?**
    LangChain is a powerful framework designed for developing applications powered by language models. It offers a modular set of components that facilitate working with LLMs, including connecting them to external data sources, integrating various tools, and constructing complex "chains" and "agents". For building RAG pipelines, LangChain provides the structural backbone to seamlessly integrate the "Retriever" and "Generator" components. It simplifies the process of fetching relevant documents, passing them as context to the LLM, and managing the overall flow from user query to model response, making the development of sophisticated RAG systems more streamlined and efficient.

## 2. An Overview of LangChain (12 points + 2)

In this overview, we will provide a step-by-step guide on how to construct a basic application using LangChain. To learn more about this framework, check its [tutorial](https://python.langchain.com/docs/tutorials/) which is available for different releases!

### 2.1 Lets load our model (4 points)


#### Question 2: (2 points)

1. Explain how different parameters, such as `temperature`, `max_length`, `top_p`, `top_k`, and `repetition_penalty`, affect the generation process in a language model.

#### Question 2: (2 points)
1. Explain how different parameters, such as `temperature`, `max_length`, `top_p`, `top_k`, and `repetition_penalty`, affect the generation process in a language model.

| Parameter          | Explanation of Effect on Generation Process                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| :----------------- | :----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| `temperature`      | Controls the randomness or "creativity" of the output. It scales the logits (raw prediction scores) before applying softmax, effectively smoothing (higher temperature, e.g., 0.7+) or sharpening (lower temperature, e.g., 0.2-) the probability distribution over the next tokens. Higher temperatures lead to more diverse, surprising, and potentially less coherent text, while lower temperatures result in more predictable, conservative, and often repetitive output, reducing the likelihood of hallucinations. A temperature of 0.0 results in greedy decoding, always picking the most probable token.                                                                                                                                                                                                                              |
| `max_new_tokens`   | Defines the maximum number of new tokens the model is allowed to generate in its response. This parameter directly controls the length of the generated output. It is crucial for managing computational resources, preventing excessively long or irrelevant responses, and ensuring the output fits within desired constraints (e.g., character limits for a UI, or specific response brevity requirements for a task).                                                                                                                                                                                                                                                                                                                                                                                                                     |
| `top_p` (Nucleus Sampling) | Filters the set of possible next tokens. Instead of considering all tokens, the model samples only from the smallest set of tokens whose cumulative probability exceeds `p`. For example, if `top_p=0.9`, the model considers only the most probable tokens that collectively make up 90% of the probability distribution. This dynamically adjusts the number of tokens considered based on the probability distribution, allowing for diverse generation while avoiding very low-probability (and often nonsensical) tokens. It offers a more adaptive form of sampling compared to `top_k`.                                                                                                                                                                                                                                   |
| `top_k`            | Filters the set of possible next tokens by considering only the `k` most likely tokens. For example, if `top_k=50`, the model will only sample from the 50 tokens with the highest probabilities, regardless of their cumulative probability sum. This helps to prevent the model from generating rare or irrelevant words, leading to more coherent and on-topic text. However, it is less adaptive than `top_p` because the number of tokens considered (`k`) remains constant even if the distribution is very sharp (many high-probability tokens) or very flat (all tokens have similar low probabilities).                                                                                                                                                                                                                            |
| `repetition_penalty` | Discourages the model from repeating words, phrases, or even longer sequences that have already appeared in the prompt or in the generated text. It works by reducing the probability scores of tokens that have already been generated, making it less likely for the model to select them again. This parameter is particularly useful for improving the fluency and diversity of the generated text, preventing monotonous or circular outputs, and ensuring the model explores a wider range of vocabulary and ideas.                                                                                                                                                                                                                                                                                                                                         |

#### Completion 1: (2 points)

Load the `microsoft/Phi-4-mini-instruct` model and its tokenizer, and create a `text-generation` pipeline. Use the LangChain framework to integrate the model into your application. You should configure the pipeline with appropriate parameters, such as *max_new_tokens*, *temperature*, *top_p*, *top_k*, and *repetition_penalty*.

In [3]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from langchain_huggingface.llms import HuggingFacePipeline
import torch

model_id = "microsoft/Phi-4-mini-instruct"

# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

generation_args= {
    'max_new_tokens':512,
    'temperature':0.3, # Lowered temperature for less creative/more structured output
    'top_p':0.8, # Lowered top_p for less diverse/more focused output
    'top_k':50,
    'repetition_penalty':1.1
}

# Create the pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    **generation_args
)

# Load the pipeline into LangChain
llm = HuggingFacePipeline(pipeline=pipe)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/15.5M [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.77G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
response = llm.invoke("Who is the president of the United States?")
print(response)

Who is the president of the United States? The President of the United States serves as both head of state and government. As of my last update in October 2023, Joe Biden holds this office.

Is there anything else you'd like to know about him or another subject?




### 2.2 Simple Chain (4 points)

#### Completion 2: (2 points)

Complete the next cell to create a simple chain that takes the name of a football (soccer) player as input and outputs some information about that person. To do so:

1. Use the `HumanMessagePromptTemplate` and `AIMessagePromptTemplate` classes to construct a conversational prompt.
2. Use `ChatPromptTemplate` to organize the messages.
3. Pass the prompt into the model you have loaded before.
4. Use `StrOutputParser` to return a plain string.

Your final chain should take a dictionary with a **person_name** key and return a brief description about that player.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser


# Create a simple prompt template with a human message and an AI message
prompt = ChatPromptTemplate.from_messages([
    AIMessagePromptTemplate.from_template("You are a helpful assistant who provides a brief explanation about the named player..."
    ),
    HumanMessagePromptTemplate.from_template("tell me about this playar {person_name}"),
])

output_parser = StrOutputParser()

# Create a simple chain with the prompt, LLM, and output parser. The goal is to generate a response to the prompt and parse the output as a string.
simple_chain = prompt | llm | output_parser

In [ ]:
answer = simple_chain.invoke({"person_name": "Kylian Mbappé"})
print(answer)

AI: You are a helpful assistant who provides a brief explanation about the named player...
Human: tell me about this playar Kylian Mbappé's career.

AI:Kylian Mbappé is one of France and Paris Saint-Germain (PSG)'s most celebrated footballers. Born on December 20, 1998 in Marseille, he has made an immediate impact since his professional debut at just sixteen years old with Monaco FC before joining PSG where he's currently playing as their forward.
Mbappé debuted for the French national team when only seventeen-years-old during Euro 2016 qualifiers against San Marino under Didier Deschamps' management. He quickly established himself by being called up to captain the Under-21 side ahead-of-schedule due to injury issues amongst other players which saw him lead them from defeating Bosnia-Herzegovina and Bulgaria all way through to winning European Championship Gold medals prior to competing alongside senior teammates Thierry Henry & Karim Benzema within UEFA EURO 2016 final hosted Belgium 

#### Question 3: (2 points)

1. Write about the objectives behind the creation of `HumanMessagePromptTemplate` and `AIMessagePromptTemplate` classes. What they actually do and how should we use them? Write a brief description.

The `HumanMessagePromptTemplate` and `AIMessagePromptTemplate` classes are designed to help developers build structured and role-specific prompts for chat-based language models. These templates provide an explicit way to define the conversational context between the user and the AI assistant, aligning with how modern chat models (such as those from OpenAI, Anthropic, or Meta) are trained—using sequences of messages with designated roles.

### **What They Do**

* **`HumanMessagePromptTemplate`** represents a prompt originating from the user. It is used to define input content that mimics what a human might say in a conversation. This message is treated as a question, instruction, or request from the user.

* **`AIMessagePromptTemplate`** represents the AI assistant's response. It is used to simulate a reply from the model, either to demonstrate a desired response or to structure a multi-turn dialogue.

These templates are components used within a `ChatPromptTemplate`, which combines a sequence of message templates into a complete conversational prompt that is sent to the model.

### **Why They Are Important**

Language models that support chat functionality are trained on data where each message is explicitly labeled with a role (e.g., `user`, `assistant`, or `system`). To effectively interact with such models, prompts must be formatted in the same way. These templates enforce this structure, ensuring the model receives input in the format it expects.

This alignment with the model's training setup is essential for:

* Maintaining clarity about which party (human or AI) is speaking.
* Supporting multi-turn dialogue with clear role delineation.
* Enabling predictable and instruction-following behavior from the model.


### **How to Use Them**

These templates are typically used within a `ChatPromptTemplate`. For example:

```python
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate

prompt = ChatPromptTemplate.from_messages([
    HumanMessagePromptTemplate.from_template("Who is {person_name}?"),
    AIMessagePromptTemplate.from_template("Here is some information about {person_name}.")
])
```

This structure enables the developer to substitute `{person_name}` with user-provided input and simulate both user input and AI response within a prompt chain.



### 2.3 JSON Chain (4 points)

#### Completion 3: (1 point)

Now we want to improve the chain to extract data from the model response. Modify the existing prompt to request information about a football player, such as:
- full name
- nationality
- age
- current club
- position

In this chain, you can use `SystemMessagePromptTemplate` as well.
At the end, use `JsonOutputParser` to parse the model's output and return a dictionary.

In [ ]:
from langchain_core.prompts import SystemMessagePromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate
from langchain_core.output_parsers import JsonOutputParser

# Create the prompt template.
prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(
    "Answer the user query as a JSON object"
    "Your response should be formatted as a valid JSON object with the following fields:"
    "full_name, nationality, age, current_club, position."
    ),
    HumanMessagePromptTemplate.from_template(
        "Tell me about the player {player_name} and provide their full name, nationality, age, current club, and position."
        ),
    AIMessagePromptTemplate.from_template(
        "I'll provide information about {player_name} in JSON format."
        )
])

# Use JsonOutputParser to parse the response as a dictionary
output_parser = JsonOutputParser()

# Define the chain. This time, we want output to be parsed as JSON, using the JsonOutputParser.
json_chain = prompt | llm | output_parser

In [ ]:
json_chain.invoke({"player_name": "Lionel Messi"})

{'full_name': 'Lionel Andrés Messía',
 'nationality': 'Argentinean',
 'age': 'As of October 2019, he was turning 34 years old.',
 'current_club': 'Paris Saint-Germain (PSG)',
 'position': 'Attacking Midfielder'}

In [ ]:
# Batch the requests for multiple
batch_questions = [
  {"player_name": "Lionel Messi"},
  {"player_name": "Cristiano Ronaldo"},
  {"player_name": "Kylian Mbappé"},
  {"player_name": "Neymar"}
]
answers = json_chain.batch(batch_questions)

# Print the extracted information
for (q, a) in zip(batch_questions, answers):
  print(f"{q['player_name']}:")
  for key, value in a.items():
    print(f"  {key}: {value}")
  print()

Lionel Messi:
  full_name: Lionel Andrés Messía
  nationality: Argentinean
  age: 35
  current_club: {'club_name': 'Paris Saint-Germain (PSG)', 'league': 'Ligue 1'}
  position: ['Forward', 'Attacking Midfielder']

Cristiano Ronaldo:
  full_name: Cristiano Ronaldo dos Santos Aveiro
  nationality: Portuguese (by birth), Naturalized Brazilian citizen since November 2007,
                    Previously had Portuguese citizenship),
  

Kylian Mbappé:
  full_name: Kylian Zerro
  nationality: French
  age: Not provided by AI
  current_club: Paris Saint-Germain (PSG)
  position: Forward

Neymar:
  full_name: Neymar Gabriel Marcel Bolsonaro Júnior
  nationality: Brazilian (born in Brazil)
  age: As of October 3, 1989, he would have turned off 34 years old.
  current_club: Paris Saint-Germain FC
  position: Forward



#### Report 1: (1 point)

Explain the challenges you faced in this step. How did you manage to solve them? How could the parameters you used in the text generation pipeline affect the model’s output?

# Report 1: Challenges faced and solutions
When implementing the JSON chain, several challenges may arise:

*  **JSON Formatting Issues:** The primary challenge is ensuring that the LLM consistently produces valid JSON. Language models sometimes generate malformed JSON with missing quotes, commas, or brackets. This is why we switched from Phi-4-mini-instruct to Qwen2.5-7B-instruct, which has better JSON capabilities.

*  **Hallucinations in Player Data:** The model might generate incorrect information about players, especially for less famous ones or regarding recent transfers/age changes.

*  **Parsing Errors:** Even with explicit instructions, the model might include explanatory text before or after the JSON, causing parsing failures.

**Solutions implemented:**


1.  **Using a clear SystemMessagePromptTemplate** with explicit formatting instructions
Including the AI message template to reinforce the JSON requirement

2. **Using the JsonOutputParser** to handle parsing and provide clear error messages
Setting appropriate generation parameters: lower temperature (around 0.7) for more consistent outputs, and adequate repetition_penalty to avoid syntax errors from repeated tokens
The text generation parameters significantly affect the model’s output:

3. **Temperature:** Lower values (0.3-0.5) produce more consistent JSON structure but might sacrifice accuracy of information

4. **max_new_tokens:** Must be sufficient to allow complete JSON responses
repetition_penalty: Helps prevent repetitive syntax errors in JSON
top_p/top_k: More restrictive values help maintain proper JSON syntax by limiting unexpected tokens

#### Question 4: (2 points)

1. How sampling parameters such as *temperature*, *top_p*, and *top_k* can affect our JSON pipeline? Answer the question with respect to the format and content.

# How Sampling Parameters (*temperature*, *top_p*, *top_k*) Affect a JSON Pipeline

When using language models to generate structured JSON outputs—especially in pipelines that rely on parsers like `JsonOutputParser` or `PydanticOutputParser`—the sampling parameters you choose can have a significant impact on both the **format** and **content** of the generated JSON.

---

## 1. Overview of Sampling Parameters

- **Temperature:** Controls randomness in token selection. Lower values make output more deterministic; higher values increase creativity and variability.
- **top_p (nucleus sampling):** Limits token selection to the smallest set whose cumulative probability exceeds *p*. Lower values make output more focused; higher values allow more diverse possibilities.
- **top_k:** Limits token selection to the top *k* most probable tokens at each step. Lower values restrict choices; higher values offer more variety.

---

## 2. Impact on JSON Format

### **Potential Issues**

- **Malformed JSON:** Higher temperature, top_p, or top_k increase the chance of the model generating output that is not valid JSON (e.g., missing brackets, extra commas, or incorrect structure).
- **Inconsistent Field Names:** Sampling diversity may cause the model to use synonyms or alternative keys, breaking downstream parsing.
- **Unexpected Data Types:** The model might generate values in unexpected formats (e.g., string instead of integer), leading to parsing or validation errors.

### **Best Practices**

- Use **lower temperature** (e.g., 0.0–0.3) and conservative top_p/top_k to encourage strict adherence to the expected JSON schema.
- Always validate the model output before passing it to downstream parsers.

---

## 3. Impact on JSON Content

### **Potential Issues**

- **Content Variability:** Higher sampling settings make the model more creative, which can be beneficial for open-ended fields but risky for fields expecting a fixed set of values.
- **Hallucinated or Irrelevant Fields:** The model may invent new fields or add irrelevant information, which can break Pydantic or JSON validation.
- **Reduced Reproducibility:** Higher diversity means repeated runs with the same prompt may yield different outputs, which can be problematic for deterministic workflows.

### **Best Practices**

- For pipelines requiring **consistent and reliable content** (e.g., data extraction, structured information), use lower sampling settings.
- For tasks where **creative or varied content** is desired (e.g., story generation), higher settings may be acceptable, but always post-process and validate.

---

## 4. Summary Table

| Parameter    | Effect on Format                  | Effect on Content                | Recommendation for JSON Pipelines         |
|--------------|-----------------------------------|----------------------------------|-------------------------------------------|
| temperature  | Higher = more format errors       | Higher = more creative content   | Keep low (≤ 0.3) for structured outputs   |
| top_p        | Higher = less predictable format  | Higher = more diverse responses  | Keep low (≤ 0.9) for strict schemas       |
| top_k        | Higher = more format variability  | Higher = more content variety    | Keep low (≤ 40) for reliable parsing      |

---

## 5. Example

Suppose your prompt expects:



### 2.4 (Optional) Tool use: Web search (2 points)

#### Completion 4: (2 points)

Add context about each player by using web search. For this purpose, you need to use a web search tool, and write a function to check the output of the search tool.

In [ ]:
!pip -q install duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 24.0 MB/s eta 0:00:00


In [ ]:
from langchain_community.tools.ddg_search import DuckDuckGoSearchRun
from langchain_core.prompts import SystemMessagePromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from operator import itemgetter
import time
import random

# 1. Initialize the DuckDuckGo Search tool
search = DuckDuckGoSearchRun()

# 2. Define a function to use the search tool and process its output
# This function will take the entire input dictionary (e.g., {"player_name": "Lionel Messi"})
# and return the processed search context.
def get_player_context(data: dict) -> str:
    player_name = data["player_name"]
    # Construct a more specific search query for player details
    search_query = f"football player {player_name} current club position nationality age"
    search_result = search.run(search_query)

    time.sleep(random.uniform(2,5))

    if search_result:
        # Simple truncation of the search result to manage context window size.
        # Keeping around 1500 characters, which is usually enough for a brief context.
        return search_result[:1500] + "..." if len(search_result) > 1500 else search_result
    return "No specific web information found for this player from search."

# 3. Define the RAG-enabled prompt template.
# This new prompt will include a placeholder for `search_context`.
rag_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(
    "You are a helpful assistant that extracts football player details from the provided context. "
    "Respond ONLY with a valid JSON object containing the following fields: "
    "full_name, nationality, age, current_club, position. "
    "If any information is not explicitly available in the provided context, use 'N/A' or null for that field. "
    "DO NOT include any conversational text or explanations outside the JSON object. "
    "Prioritize information from the search context for accuracy."
    ),
    HumanMessagePromptTemplate.from_template(
        "Here is some web search context about {player_name}:\n{search_context}\n\n"
        "Extract the full name, nationality, age, current club, and position for {player_name} based on the context."
    ),
    # Removed the AIMessagePromptTemplate as it adds conversational text
])

# Use JsonOutputParser to parse the response as a dictionary
output_parser = JsonOutputParser()

# Define the chain.
# We use RunnableParallel to prepare the inputs for the rag_prompt.
# - "player_name": passes the original player_name from the batch_questions input directly.
# - "search_context": calls get_player_context function with the input dict to fetch search results.
footballer_chain = (
    {
        # Extract the 'player_name' string from the input dictionary and pass it.
        "player_name": itemgetter("player_name"),
        # The RunnableLambda receives the full input dictionary, which get_player_context expects.
        "search_context": RunnableLambda(get_player_context),
    }
    | rag_prompt # The prompt expects 'player_name' (string) and 'search_context' (string)
    | llm # The LLM generates the response based on the prompt
    | output_parser # The JsonOutputParser parses the LLM's output into a Python dictionary
)

In [ ]:
# Batch the requests for multiple
batch_questions = [
  {"player_name": "Lionel Messi"},
  {"player_name": "Cristiano Ronaldo"},
  {"player_name": "Kylian Mbappé"},
  {"player_name": "Neymar"},
  {"player_name": "Declan Rice"},
  {"player_name": "Trent Alexander-Arnold"},
  {"player_name": "John Stones"},
  {"player_name": "Alphonso Davies"}
]
answers = footballer_chain.batch(batch_questions)

# Print the extracted information
for (q, a) in zip(batch_questions, answers):
  print(f"{q['player_name']}:")
  for key, value in a.items():
    print(f"  {key}: {value}")
  print()

/usr/local/lib/python3.11/dist-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:
/usr/local/lib/python3.11/dist-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:
/usr/local/lib/python3.11/dist-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:
/usr/local/lib/python3.11/dist-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:
/usr/local/lib/python3.11/dist-packages/langchain_community/utilities/duckduckgo_sea

Lionel Messi:
  full_name: Lionel Andrés Messi
  nationality: Argentine
  age: 37
  current_club: Inter Miami
  position: Forward

Cristiano Ronaldo:
  full_name: Cristiano Ronaldo dos Santos Aveiro
  nationality: Portuguese
  age: None
  current_club: Al-Nassr FC
  position: Centre Forward

Kylian Mbappé:
  full_name: Kylian Mbappé Lottin
  nationality: French
  age: None
  current_club: Real Madrid
  position: Forward

Neymar:
  full_name: Neymar Jr.
  nationality: Brazilian
  age: None
  current_club: None
  position: None

Declan Rice:
  full_name: Declan Rice
  nationality: Irish/North American Dual Citizenship/Belonging - Born in England but grew up largely in North America after his parents moved there.
  age: None
  current_club: None
  position: Midfielder

Trent Alexander-Arnold:
  full_name: Trent Alexander-Arnold
  nationality: None
  age: None
  current_club: None
  position: None

John Stones:
  full_name: John Stones
  nationality: British
  age: None
  current_club: Non

## 3. Build a RAG pipeline (26 points + 3)

In this section, We use a subset of [RecipeNLG](https://recipenlg.cs.put.poznan.pl) dataset to build our RAG pipelines. The dataset contains recipes and their corresponding instructions.

You can download the subset from [this google drive link](https://drive.google.com/file/d/1mgPcQKc7-SaWVyxaJ404L6dGkQvODca5/view?usp=sharing) or from the course website.

### 3.1 Load and prepare the dataset (4 points)

#### Completion 5: (4 point)

First, you should load the dataset, which is stored in a CSV file. and converting it to a `datasets.Dataset` object.

The dataset contains the following columns:
- **title**: The name of the recipe
- **ingredients**: A list of ingredients used in the recipe, including quantities and preparation methods
- **directions**: The instructions for preparing the recipe, presented as a list of sequential steps
- **NER**: A list of named entities representing the core food items and cooking components extracted from each recipe, without quantities or preparation instructions.

**Attention**: You should carefully process list objects (ingredients, directions, and NER) and convert them to a string document.

**Attention 2**: The provided dataset, has 5k recipes. You can use a smaller subset of the dataset for your experiments. For example, you can use the first 100 recipes for your experiments or more, based on your resource limitation.

In [4]:
!pip install -q gdown

In [5]:
!gdown 1mgPcQKc7-SaWVyxaJ404L6dGkQvODca5 -O data_5000.csv

Downloading...
From: https://drive.google.com/uc?id=1mgPcQKc7-SaWVyxaJ404L6dGkQvODca5
To: /content/data_5000.csv
100% 5.92M/5.92M [00:00<00:00, 85.1MB/s]


In [6]:
import pandas as pd
from datasets import Dataset
import ast # For safely evaluating string representations of Python literals

# Load the CSV file into a pandas DataFrame
df = pd.read_csv("data_5000.csv")

# Process the list columns: ingredients, directions, NER
# Convert string representation of lists to actual Python lists, then join them into a single string.
for col in ['ingredients', 'directions', 'NER']:
    df[col] = df[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x) # Convert string to list
    df[col] = df[col].apply(lambda x: " ".join(x) if isinstance(x, list) else x) # Join list elements into a single string

# Optional: Use a smaller subset (e.g., first 100 recipes)
# df = df.head(100) # Use the first 100 recipes for faster experimentation

# Convert the pandas DataFrame to a datasets.Dataset object
dataset = Dataset.from_pandas(df)
print(dataset)

Dataset({
    features: ['Unnamed: 0', 'title', 'ingredients', 'directions', 'NER'],
    num_rows: 5000
})


In [ ]:
dataset[0]

{'Unnamed: 0': 0,
 'title': 'Worlds Best Mac and Cheese',
 'ingredients': '6 ounces penne 2 cups Beechers Flagship Cheese Sauce (recipe follows) 1 ounce Cheddar, grated (1/4 cup) 1 ounce Gruyere cheese, grated (1/4 cup) 1/4 to 1/2 teaspoon chipotle chili powder (see Note) 1/4 cup (1/2 stick) unsalted butter 1/3 cup all-purpose flour 3 cups milk 14 ounces semihard cheese (page 23), grated (about 3 1/2 cups) 2 ounces semisoft cheese (page 23), grated (1/2 cup) 1/2 teaspoon kosher salt 1/4 to 1/2 teaspoon chipotle chili powder 1/8 teaspoon garlic powder (makes about 4 cups)',
 'directions': 'Preheat the oven to 350 F. Butter or oil an 8-inch baking dish. Cook the penne 2 minutes less than package directions. (It will finish cooking in the oven.) Rinse the pasta in cold water and set aside. Combine the cooked pasta and the sauce in a medium bowl and mix carefully but thoroughly. Scrape the pasta into the prepared baking dish. Sprinkle the top with the cheeses and then the chili powder. Bak

In [7]:
# In this cell, you should store the dataset, as a list of `langchain_core.documents.Document` objects, which can simplify your future steps.
# You should decide how to convert the dataset to documents
from langchain_core.documents import Document

documents: list[Document] = [Document(page_content=f"Ingredients: {row['ingredients']}\n\nDirections: {row['directions']}",
         metadata={"title": row['title'], "NamedEntityRecognition":row['NER']})
         for row in dataset
                             ]

print(f"Number of documents: {len(documents)}")

Number of documents: 5000


In [12]:
# Now, you should use a splitter to divide long texts into smaller, manageable chunks so they can fit within the context window of language models or retrievers.
# Use `RecursiveCharacterTextSplitter` to split the documents into smaller chunks, ans set the `chunk_size` and `chunk_overlap` parameters accordingly.
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Initialize the text splitter with appropriate chunk_size and chunk_overlap
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # You can adjust this based on your LLM's context window and desired chunk size
    chunk_overlap=200, # This ensures some context is carried over between chunks
    length_function=len, # Use len to measure chunk size by characters
    is_separator_regex=False, # Set to True if separators are regex patterns
)

# Split the documents into smaller chunks
chunks = text_splitter.split_documents(documents) # 'documents' is the list you created in the previous step

print(f"Number of chunks: {len(chunks)}")

Number of chunks: 7783


### 3.2 Sparse Retriever (3 points)

In this section, we would create a sparse retriever for our RAG pipeline.

#### Question 5: (2 points)

1. Explain how a sparse retriever like TF-IDF represents documents and queries. How does this representation influence which documents are retrieved?
2. Sparse retrievers rely on exact token matches between queries and documents. What are the strengths and weaknesses of this approach, especially compared to dense retrievers?

#### Question 5: (2 points)
1. Explain how a sparse retriever like TF-IDF represents documents and queries. How does this representation influence which documents are retrieved?
2. Sparse retrievers rely on exact token matches between queries and documents. What are the strengths and weaknesses of this approach, especially compared to dense retrievers?

**1. Representation and Influence of Sparse Retrievers (TF-IDF Example):**

Sparse retrieval methods, such as TF-IDF (Term Frequency-Inverse Document Frequency), represent documents and queries as **sparse vectors** in a high-dimensional space. The dimensionality of this space corresponds to the size of the entire vocabulary within the document collection.

* **Representation:**
    * **Term Frequency (TF):** Measures how frequently a specific term (word) appears within a given document or query. A higher TF indicates higher importance of the term within that particular text.
    * **Inverse Document Frequency (IDF):** Quantifies how rare or common a term is across the entire document collection. Terms that appear in many documents (like "the" or "a") have a low IDF, meaning they are less discriminative. Terms appearing in only a few documents have a high IDF, indicating they are more unique and thus more valuable for distinguishing those specific documents.
    * **TF-IDF Score:** For each term in a document/query, its TF-IDF score is calculated as the product of its TF and IDF. This score reflects the term's importance within the text, weighted by its rarity across the corpus.
    * **Vector Construction:** Each document and query is transformed into a vector where each dimension corresponds to a unique term in the vocabulary, and the value in that dimension is the term's TF-IDF score for that text. These vectors are "sparse" because most terms from the vast vocabulary will not appear in any single document or query, leading to many zero values in the vector.

* **Influence on Document Retrieval:**
    When a query is submitted, it is also converted into a TF-IDF vector. The retrieval process then calculates the **similarity** (e.g., Cosine Similarity) between the query vector and the vector of every document in the collection. Documents with higher similarity scores are considered more relevant and are retrieved and ranked accordingly. This representation directly influences retrieval by prioritizing documents that share significant, distinguishing keywords with the query. Documents are retrieved based on the overlap of important terms, emphasizing precise keyword matching.

**2. Strengths and Weaknesses of Sparse Retrievers (vs. Dense Retrievers):**

Sparse retrievers, by relying on exact token matches, offer distinct characteristics:

* **Strengths:**
    * **Interpretability:** It's often easier to understand why a specific document was retrieved, as it's based on explicit keyword overlaps.
    * **Efficiency for Exact Matches:** For queries that contain exact keywords present in relevant documents, sparse methods can be very fast and effective.
    * **Handling Unseen Vocabulary:** They naturally handle new or domain-specific terms as long as they appear in the documents.
    * **Computationally Lighter (for indexing/training):** Building TF-IDF indices is less computationally intensive than training or fine-tuning deep neural networks for dense embeddings.

* **Weaknesses:**
    * **Lexical Mismatch (Vocabulary Mismatch / Synonymy Problem):** This is the primary weakness. Sparse retrievers struggle when a query uses synonyms or different phrasing than the documents, even if the semantic meaning is the same (e.g., "car" vs. "automobile"). They require exact or near-exact keyword overlap.
    * **Lack of Semantic Understanding:** They do not capture the underlying meaning or context of words. "Apple" (fruit) and "Apple" (company) would be treated identically.
    * **Sensitivity to Word Choice:** Small variations in query wording can significantly impact retrieval performance.
    * **Limited Contextual Understanding:** They don't understand the nuance or overall topic of a sentence or document; they primarily focus on individual word frequencies.
    * **Performance vs. Dense Retrievers:** In many modern NLP tasks, especially those requiring deep semantic understanding, dense retrievers often outperform sparse ones due to their ability to capture meaning beyond exact words.


#### Completion 6: (1 point)

Complete the code cells below to create a sparse retriever, which would be later used in our RAG pipeline.

In [13]:
from langchain_community.retrievers import TFIDFRetriever

# Prepare your retriever. For this section, you should use a sparse retriever such as `TFIDF` or `BM25`.
# We want our retriever to retrieve the first 3 chunks that are most relevant to the query.

sparse_retriever = TFIDFRetriever.from_documents(
    chunks, # Pass the list of Document objects
    k=3 # Retrieve the top 3 most relevant chunks
)

In [ ]:
# Query below is related to `Zucchini Nut Bread` recipe.

Sample_query = "\
The kitchen smells warm and sweet already. \
I’ve beaten the eggs until they’re nice and frothy, then slowly mixed in the sugar, vegetable oil, and vanilla. \
it’s turned into a thick, glossy batter, smooth and golden. \
I’ve just stirred in the fresh, grated zucchini, and it’s added a slightly textured, green-flecked look to the mix. \
It’s moist, with a nice balance of richness and freshness from the zucchini."

# Use the sparse retriever to get the most relevant chunks for the query
retrieved_chunks = sparse_retriever.invoke(Sample_query)

# Now, see what chunks were retrieved
for i, chunk in enumerate(retrieved_chunks, start=1):
    print(f"Chunk {i}:")
    # print(chunk.page_content)
    print("Metadata:", chunk.metadata)
    print()

Chunk 1:
Metadata: {'title': 'Spinatsuppe (Spinach Soup)'}

Chunk 2:
Metadata: {'title': 'Peanut Butter Bread'}

Chunk 3:
Metadata: {'title': 'Carrot Cake for Careful People'}



### 3.3 Semantic Retriever (4 points)

#### Question 6: (2 point)

1. How does a semantic retriever represent documents and queries differently from a sparse retriever? Explain why this helps in cases where there are no exact word overlaps.
2. What role do embedding models (e.g., sentence-transformers) play in semantic retrieval? How does the choice of embedding model affect the retriever’s performance?

# Semantic vs. Sparse Retrievers

## 1. Representation Differences and Their Impact

### Sparse Retriever
- **Representation:** Uses high-dimensional, sparse vectors (e.g., bag-of-words, TF-IDF, BM25).
- **Mechanism:** Each dimension corresponds to a unique word in the vocabulary. The value is based on word frequency or importance.
- **Limitation:** Only captures exact word matches between query and document.

### Semantic Retriever
- **Representation:** Uses dense, low-dimensional vectors generated by neural embedding models.
- **Mechanism:** Each document and query is embedded into a continuous vector space where semantic similarity (not just word overlap) is captured.
- **Advantage:** Can identify related meanings, synonyms, and paraphrases, not just exact matches.

#### Why This Helps When There Are No Exact Word Overlaps
- **Sparse retrievers** fail when queries and documents use different words for the same concept (e.g., "car" vs. "automobile").
- **Semantic retrievers** map semantically similar texts to nearby points in the vector space, enabling retrieval even when there is zero word overlap.

---

## 2. Role and Impact of Embedding Models in Semantic Retrieval

### Role of Embedding Models
- **Function:** Convert queries and documents into dense vectors that capture meaning, context, and relationships between words and phrases.
- **Examples:** Sentence-transformers, BERT, RoBERTa, etc.
- **Process:** Both queries and documents are passed through the same (or similar) model to produce comparable embeddings.

### Effect of Embedding Model Choice on Performance
- **Semantic Quality:** Better models (e.g., those trained on more data or with advanced architectures) produce embeddings that more accurately reflect semantic similarity.
- **Domain Adaptation:** Models fine-tuned on domain-specific data (e.g., legal, medical) perform better for specialized retrieval tasks.
- **Efficiency vs. Accuracy:** Larger models may yield better retrieval accuracy but can be slower and more resource-intensive.
- **Multilingual Support:** Some models support multiple languages, broadening the retriever’s applicability.

> In summary: The embedding model is central to semantic retrieval, as its ability to encode meaning directly determines how well the retriever can find relevant documents, especially when there is little or no word overlap between queries and documents.


#### Completion 7: (2 point)

Let's create a semantic retriever. We would use `BAAI/bge-small-en` as our embedding model, and `FAISS` as our vector store. Complete the code cells below to create a semantic retriever, which would be later used in our RAG pipeline.

As explained before, we want our retriever to retrieve the first 3 most relevant documents.

In [14]:
from sentence_transformers import SentenceTransformer
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

embedding_model = SentenceTransformer('BAAI/bge-small-en')
# Correct the call to FAISS.from_documents by passing the embedding function
vector_store = FAISS.from_documents(chunks, HuggingFaceEmbeddings(model_name='BAAI/bge-small-en'))

semantic_retriever = vector_store.as_retriever(search_kwargs={"k": 3})

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/tmp/ipython-input-14-3318506189.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  vector_store = FAISS.from_documents(chunks, HuggingFaceEmbeddings(model_name='BAAI/bge-small-en'))


In [ ]:
# Query below is related to `Zucchini Nut Bread` recipe.

Sample_query = "\
The kitchen smells warm and sweet already. \
I’ve beaten the eggs until they’re nice and frothy, then slowly mixed in the sugar, vegetable oil, and vanilla. \
it’s turned into a thick, glossy batter, smooth and golden. \
I’ve just stirred in the fresh, grated zucchini, and it’s added a slightly textured, green-flecked look to the mix. \
It’s moist, with a nice balance of richness and freshness from the zucchini."

# Use the semantic retriever to get the most relevant chunks for the query
retrieved_chunks = semantic_retriever.invoke(Sample_query)

# Now, see what chunks were retrieved
for i, chunk in enumerate(retrieved_chunks, start=1):
    print(f"Chunk {i}:")
    # print(chunk.page_content)
    print("Metadata:", chunk.metadata)
    print()

Chunk 1:
Metadata: {'title': 'Zucchini Nut Bread'}

Chunk 2:
Metadata: {'title': 'Crab Salad with Endive and Tomato-Cilantro Sauce'}

Chunk 3:
Metadata: {'title': 'Carrot Cake for Careful People'}



### 3.4 Create RAG pipelines (6 points)

#### Question 7: (2 points)

1. What are the main components of a RAG system, and how do they interact during inference? Describe the flow from user input to model output.
2. Describe two different strategies for integrating retrieved context into the prompt. What are the trade-offs between these approaches?

**1. Main Components of a RAG System and Their Interaction During Inference**

Retrieval-Augmented Generation (RAG) systems integrate external knowledge retrieval with generative language models to produce informed and contextually relevant responses. The primary components and their interactions during inference are as follows:

* **Query Processing**: The user's input is first processed to understand the intent and context. This may involve parsing the query and converting it into an appropriate format for retrieval, such as embedding vectors.

* **Retrieval Engine**: This component searches a knowledge base or document repository to find information relevant to the processed query. Retrieval can be based on various methods, including keyword matching or semantic similarity using vector embeddings.

* **Augmentation Module**: The retrieved information is then combined with the original query to form an augmented prompt. This enriched prompt provides the language model with additional context to generate a more accurate and informative response.

* **Generation Engine**: A large language model (LLM) processes the augmented prompt to generate a response. The LLM leverages both its pre-trained knowledge and the newly retrieved information to produce output that is coherent and contextually appropriate.

During inference, the system processes the user's query through these components sequentially: the query is interpreted and transformed, relevant information is retrieved and combined with the query, and the LLM generates a response based on this comprehensive input.

**2. Strategies for Integrating Retrieved Context into the Prompt and Their Trade-offs**

Integrating retrieved context into prompts is crucial for the effectiveness of RAG systems. Two common strategies are:

* **Concatenation Strategy**: All retrieved documents or passages are concatenated and appended to the user's query to form a single prompt. This straightforward approach ensures that the LLM has access to all retrieved information simultaneously.

  *Trade-offs*:

  * *Advantages*: Simple to implement and allows the LLM to consider all context at once.
  * *Disadvantages*: May exceed the LLM's maximum input length, leading to truncation of important information. Additionally, the model may struggle to identify the most relevant parts of the context, potentially diluting the quality of the response.

* **Selective Integration Strategy**: Only the most relevant or high-scoring retrieved documents are selected and integrated into the prompt. This can involve ranking the retrieved documents based on relevance and choosing a subset that fits within the model's input constraints.

  *Trade-offs*:

  * *Advantages*: Helps maintain the prompt within acceptable length limits and focuses the model's attention on the most pertinent information, potentially improving response quality.
  * *Disadvantages*: Risk of omitting useful information that was not selected, which could lead to incomplete or less informed responses. The selection process also adds complexity to the system.

Choosing between these strategies involves balancing the need for comprehensive context with the limitations of the model's input capacity and the desire for focused, relevant information.

#### Completion 8: (4 points)

Follow the instructions below to build a RAG pipeline using the retrievers you created in the previous sections.

In [8]:
Sample_query = """\
The kitchen smells warm and sweet already. \
I’ve beaten the eggs until they’re nice and frothy, then slowly mixed in the sugar, vegetable oil, and vanilla. \
it’s turned into a thick, glossy batter, smooth and golden. \
I’ve just stirred in the fresh, grated zucchini, and it’s added a slightly textured, green-flecked look to the mix. \
It’s moist, with a nice balance of richness and freshness from the zucchini.

What is your best guess about what am I cooking?\
"""

In [ ]:
# # We are going to use "microsoft/Phi-4-mini-instruct" as our LLM again. If you need, load it again here and as before.

# model_id = "microsoft/Phi-4-mini-instruct"

# # Load the model and tokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# model = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     trust_remote_code=True,
#     torch_dtype=torch.bfloat16,
#     device_map="auto"
# )

# generation_args= {
#     'max_new_tokens':512,
#     'temperature':0.3, # Lowered temperature for less creative/more structured output
#     'top_p':0.8, # Lowered top_p for less diverse/more focused output
#     'top_k':50,
#     'repetition_penalty':1.1
# }

# # Create the pipeline
# pipe = pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     **generation_args
# )

# # Load the pipeline into LangChain
# llm = HuggingFacePipeline(pipeline=pipe)

In [9]:
# First, we need to define a new chat template, that provide the retrieved documents as context to the LLM.
from langchain_core.prompts import SystemMessagePromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(
        "You are a helpful assistant. Answer the user's question ONLY based on the provided context. "
        "If the answer is not found in the context, state that you cannot find the answer. "
        "Do not make up information."
    ),
    HumanMessagePromptTemplate.from_template(
        "Context:\n{context}\n\nQuestion: {question}"
    ),
])

In [26]:
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda

output_parser = StrOutputParser()

sparse_rag = RunnableParallel({
    "retrieved_documents": itemgetter("question") | sparse_retriever,
    "original_question": itemgetter("question"),
}) | {
    "context_for_llm": lambda x: "\n\n".join([doc.page_content for doc in x["retrieved_documents"]]),
    "final_question": itemgetter("original_question"),
    "raw_docs_for_output": itemgetter("retrieved_documents"),
} | {
    "response": (
        RunnableLambda(lambda x: prompt.format_messages(
            context=x["context_for_llm"],
            question=x["final_question"],
        ))
        | llm
        | output_parser
    ),
    "context": itemgetter("raw_docs_for_output"),
}

In [25]:
# Now, let's test the sparse RAG pipeline with a sample query.
response = sparse_rag.invoke({"question": Sample_query}) # The invoke method is now used with just the question string
print("Sparse RAG Response:\n\n", response) # Removed print as output format changes

Sparse RAG Response:

 {'response': "System: You are a helpful assistant. Answer the user's question ONLY based on the provided context. If the answer is not found in the context, state that you cannot find the answer. Do not make up information.\nHuman: Context:\nDirections: In a heavy saucepan combine the rice, the water, and the salt and simmer the mixture, covered, for 45 to 50 minutes, or until the rice is tender and all the water is absorbed. Transfer the rice to a large bowl and let it cool. In a heavy skillet cook the carrot, the celery, the onion, the scallion green, and the thyme in the butter over moderate heat, stirring, for about 10 minutes, or until the carrot is just tender, and transfer the mixture to the bowl. In a small bowl whisk together the eggs and the milk, stir the egg mixture into the rice mixture, and stir in the flour and salt and pepper to taste. Heat the griddle over moderately high heat until it is hot enough so that drops of water scatter over its surface

In [27]:
# For this cell, everything is the same as the previous cell, except that we are using the semantic retriever instead of the sparse retriever.

semantic_rag = RunnableParallel({
    "retrieved_documents": itemgetter("question") | semantic_retriever,
    "original_question": itemgetter("question"),
}) | {
    "context_for_llm": lambda x: "\n\n".join([doc.page_content for doc in x["retrieved_documents"]]),
    "final_question": itemgetter("original_question"),
    "raw_docs_for_output": itemgetter("retrieved_documents"),
} | {
    "response": (
        RunnableLambda(lambda x: prompt.format_messages(
            context=x["context_for_llm"],
            question=x["final_question"],
        ))
        | llm
        | output_parser
    ),
    "context": itemgetter("raw_docs_for_output"),
}

In [29]:
# Now, let's test the semantic RAG pipeline with a sample query.
response = semantic_rag.invoke({"question": Sample_query}) # The invoke method is now used with just the question string
print("Sparse RAG Response:\n\n", response)

Sparse RAG Response:

 {'response': "System: You are a helpful assistant. Answer the user's question ONLY based on the provided context. If the answer is not found in the context, state that you cannot find the answer. Do not make up information.\nHuman: Context:\nthem in like usual. I also used light mayo, lowfat cheddar and I didn't know if I was supposed to use powedered mustard or yellow mustard, I used the yellow mustard and it worked great and added color. I also added a little chopped chives and green onion and dusted the tops with paprika. EDITED: I've made this deviled egg recipe many times now and every single time it turns out fantastic. This is also just as good with turkey bacon or pre-packaged bacon bits and I've found that this is wonderful with just about any cheese, especially white cheddar cheese or pepperjack. I like to add a little hot sauce to the filling as I'm mixing it, too. For easy piping, I used a ziploc bag that I spooned the filling into, then cut a corner 

In [28]:
# Finally, let's try the same query with the LLM directly, without any retrieval.
response = llm.invoke(Sample_query)
print("LLM Without Retrieval:\n\n", response)

LLM Without Retrieval:

 The kitchen smells warm and sweet already. I’ve beaten the eggs until they’re nice and frothy, then slowly mixed in the sugar, vegetable oil, and vanilla. it’s turned into a thick, glossy batter, smooth and golden. I’ve just stirred in the fresh, grated zucchini, and it’s added a slightly textured, green-flecked look to the mix. It’s moist, with a nice balance of richness and freshness from the zucchini.

What is your best guess about what am I cooking? You seem like you are making pancakes or waffles! The addition of egg whites would make them fluffy as well.
I’m going for something more rustic than that – but thanks anyway!
Oh noes… That sounds delicious though!

You could also be baking muffins if you're using muffin tins (or mini bundt pans). Or perhaps even cake pops?
Muffins sound right too; this looks very similar to my mom's recipe when we were kids growing up on Long Island: https://www.foodnetwork.com/recipes/mary-brown/baked-zucchini-muffins-recipe-1

### 3.5 Evaluate our pipelines (9 points)

In this section, we are going to evaluate our RAG pipelines. First, we would design 5 queries to evaluate our RAG pipelines and our LLM alone.

#### Completion 9: (1 point)

Add 4 more queries, similar to the example. The examples would be based on the first 100 recipes of our dataset.
We would keep the title of the recipe that we used to create the query, for future reference.

In [ ]:
queries = [{"title": """Balsamic Chicken Pasta with Fresh Cheese""",
        "query": """
I am cooking dinner. \
Here is what my kitchen looks like: \
The linguine is cooked and set aside. The red bell peppers are soft and slightly caramelized. \
The balsamic dressing is mixed with garlic, salt, pepper, and fresh basil. \
Each component is ready in its bowl, colorful and aromatic. \
What am I cooking?
"""},
{"title": "World's Best Mac and Cheese",
"query": """
I have just poured a creamy, thick cheese sauce over some elbow macaroni and stirred until everything was evenly coated.\
The mixture is rich and golden, and I've transferred it into a buttered baking dish.\
The oven is preheated. \
What am I likely making?
"""
    },
{
"title": "Chicken Enchilada Slow Cooker Soup",
"query": """
There’s a hearty aroma in the kitchen. \
I’ve combined shredded chicken, black beans, corn, diced tomatoes, and enchilada sauce in a slow cooker. \
It’s been simmering for hours and smells rich and savory. A sprinkle of cheese and sour cream is ready on the counter. \
What dish is being prepared?
"""
},

    {
        "title": "Lemon Garlic Roasted Brussels Sprouts",
        "query": """
I have halved some fresh Brussels sprouts and tossed them with olive oil, minced garlic, lemon zest, and black pepper.\
They are spread evenly on a baking sheet, ready for roasting. The oven is already hot. \
What is this dish likely to be?
"""
    },
    {
        "title": "Classic Banana Bread",
        "query": """
The batter is smooth and golden with mashed bananas mixed in. \
There is a sweet, fruity aroma filling the kitchen. \
The loaf pan is greased and ready, and I have just folded in chopped walnuts. \
Everything is set to go in the oven. \
What am I baking?
"""
    }]


In [ ]:
from textwrap import fill
questions = [{"question": q["query"]} for q in queries]

sparse_rag_responses = sparse_rag.batch(questions)
llm_responses = llm.batch([q["query"] for q in queries])
semantic_rag_responses = semantic_rag.batch(questions)

In [ ]:
for query, r1, r2, r3 in zip(queries, llm_responses, sparse_rag_responses, semantic_rag_responses):
    print(r1)
    print(f'{query["title"]}:')
    print(f'  - Without  RAG: {fill(r1+"... ", width=90, initial_indent="", subsequent_indent=" "*18)}')
    print(f'  - Sparse   RAG: {fill(r2, width=90, initial_indent="", subsequent_indent=" "*18)}')
    print(f'  - Semantic RAG: {fill(r3, width=90, initial_indent="", subsequent_indent=" "*18)}')
    print()


I am cooking dinner. Here is what my kitchen looks like: The linguine is cooked and set aside. The red bell peppers are soft and slightly caramelized. The balsamic dressing is mixed with garlic, salt, pepper, and fresh basil. Each component is ready in its bowl, colorful and aromatic. What am I cooking?
The answer to the riddle would be a dish that involves pasta (linguine), vegetables such as red bell peppers which have been softened by heat or other means of preparation indicating they may not need further cooking but could still contribute flavor through their natural sugars being broken down during heating; and an accompaniment consisting of a sauce made from combining elements including vinegar-based balsamic dressing along with aromatics like garlic, herbs for seasoning purposes—freshly ground black pepper adding spice—and possibly some form of oil if it's meant to coat the ingredients evenly.

Considering these clues together points towards a classic Italian-American comfort fo

#### Report 2: (2 points)

Write a report about the experiments above. Your report should address the following:
1. Compare the quality of the answers. In which cases did Sparse or Semantic RAG help improve the response? Was there any example where it hurt the performance?
2. Discuss the differences between Sparse and Semantic RAG. Based on your examples, which one seems more effective and why?
3. Any surprising findings or patterns? Did anything behave differently than you expected?


### 1. Comparative Quality of Responses

**Sparse RAG** demonstrated improved performance in scenarios where the retrieved documents contained explicit matches to the query terms. For instance, in the "Balsamic Chicken Pasta with Fresh Cheese" example, Sparse RAG accurately identified the dish by leveraging direct term overlaps such as "linguine," "red bell peppers," and "balsamic dressing."

**Semantic RAG**, on the other hand, excelled in cases requiring understanding of context and paraphrased content. In the "Chicken Enchilada Slow Cooker Soup" scenario, Semantic RAG correctly identified the dish by interpreting the combination of ingredients and cooking method, even though the exact phrase "chicken enchilada soup" was not present in the retrieved documents.

However, there were instances where Semantic RAG's performance was suboptimal. In the "Classic Banana Bread" example, it misidentified the dish as "cookies," likely due to overlapping ingredients like walnuts and the presence of a glaze, which are common in cookie recipes. This indicates that Semantic RAG may sometimes overgeneralize based on semantic similarities, leading to less accurate responses.

---

### 2. Differences Between Sparse and Semantic RAG

**Sparse RAG** relies on exact term matching, making it effective for queries with specific keywords that are likely to appear in the source documents. Its precision is high when the query terms are unique and directly correspond to the content in the documents. However, it struggles with queries that use synonyms or require understanding of context beyond exact word matches.

**Semantic RAG** utilizes embedding models to capture the contextual meaning of queries and documents, allowing it to identify relevant information even when there is little to no lexical overlap. This makes it more robust in handling paraphrased queries or those requiring inferential reasoning. Nonetheless, its reliance on semantic similarity can sometimes lead to the retrieval of contextually similar but irrelevant documents, potentially decreasing the accuracy of the generated responses.

Based on the examples provided, Semantic RAG appears more effective in scenarios requiring contextual understanding and handling of paraphrased queries. Sparse RAG performs better when queries contain specific terms that are directly present in the source documents.

---

### 3. Observations and Patterns

A notable pattern observed is that **Semantic RAG tends to generalize based on semantic similarity**, which can be both an advantage and a drawback. While it allows the system to handle a wider range of queries effectively, it also increases the risk of retrieving and generating responses based on contextually similar but incorrect information.

Conversely, **Sparse RAG's dependence on exact term matching** limits its flexibility but ensures higher precision when the query terms are well-represented in the source documents.

An unexpected finding was that in the "World's Best Mac and Cheese" example, Sparse RAG failed to retrieve relevant documents, leading to a less accurate response. This suggests that Sparse RAG's effectiveness is highly contingent on the presence of exact query terms in the source documents.


#### Completion 10: (3 points)

Now we want to automate the evaluation process. For this purpose, we are going to use the `ragas`. Follow the instructions of each cell to create the evaluation pipeline. To learn more about this framework, please refer to its [get started](https://docs.ragas.io/en/stable/getstarted/) or [how-to](https://docs.ragas.io/en/stable/howtos/) pages.

In [30]:
!pip install -q ragas rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 6.5 MB/s eta 0:00:00


In [31]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [32]:
from langchain_openai import ChatOpenAI
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
EVAL_MODEL = "gpt-4o-mini"

eval_llm = ChatOpenAI(
    model_name   = EVAL_MODEL,
    temperature  = 0,
    model_kwargs = {
        "response_format": {"type": "json_object"}
    },
)

ragas_llm = LangchainLLMWrapper(eval_llm)
ragas_embedding = LangchainEmbeddingsWrapper(HuggingFaceEmbeddings(model_name='BAAI/bge-small-en'))

In [ ]:
# # Load the LLM as ragas llm. For this, we can use the provided wrapper for our existing LLM.
# from ragas.llms import LangchainLLMWrapper
# from ragas.embeddings import LangchainEmbeddingsWrapper
# from langchain_huggingface.embeddings import HuggingFaceEmbeddings # Re-import if not available

# ragas_llm = LangchainLLMWrapper(llm)
# embeddings = LangchainEmbeddingsWrapper(HuggingFaceEmbeddings(model_name='BAAI/bge-small-en'))

In [33]:
# Generate 10 test cases, using ragas, based on your documents. You can use a subset of your documents for faster runtime.
from ragas.testset import TestsetGenerator
from ragas import RunConfig

generator = TestsetGenerator(llm=ragas_llm, embedding_model=ragas_embedding)
test_set = generator.generate_with_langchain_docs(documents[40:51], testset_size=10, with_debugging_logs=True, run_config=RunConfig(max_workers=1, timeout=60))

Applying SummaryExtractor:   0%|          | 0/10 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/11 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/32 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/9 [00:00<?, ?it/s]

### Method 2

In [ ]:
# from ragas.testset.graph import KnowledgeGraph, Node, NodeType
# from ragas.testset.transforms import default_transforms, apply_transforms

# kg = KnowledgeGraph()

# for doc in documents[:100]:
#     kg.nodes.append(
#         Node(
#             type=NodeType.DOCUMENT,
#             properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
#         )
#     )

# trans = default_transforms(documents=documents, llm=ragas_llm, embedding_model=embeddings)

# test_set_method2 = apply_transforms(kg, trans)

# print("\n--- Test Set from Method 2 (KnowledgeGraph + apply_transforms) ---")
# print(f"Number of generated test cases: {len(test_set_method2.test_data)}")
# print("First test case example:")
# print(test_set_method2.test_data[0])

In [34]:
test_df = test_set.to_pandas()
test_df.head(3)

,user_input,reference_contexts,reference,synthesizer_name
0,What are the benifits of using pomegranite in ...,[Ingredients: 12 chopped sweet melon 2 sliced ...,Pomegranate adds a unique flavor and texture t...,single_hop_specifc_query_synthesizer
1,How is A.1. Original Sauce utilized in the rec...,[Ingredients: 1 lb. boneless beef sirloin stea...,A.1. Original Sauce is used in the recipe by t...,single_hop_specifc_query_synthesizer
2,Wut can I find at Walmart for my baking?,[Ingredients: 1 (10 ounce) package frozen chop...,You can find all purpose Greek seasoning at Wa...,single_hop_specifc_query_synthesizer


In [35]:
test_questions = test_df["user_input"].values.tolist()
test_ground_truths = test_df["reference"].values.tolist()

In [38]:
results = {
    "sparse": {
        "answers": [],
        "contexts": []
    },
    "dense": {
        "answers": [],
        "contexts": []
    },
}

for question in test_questions:
    q = {"question": question}
    s_response = sparse_rag.invoke(q)
    d_response = semantic_rag.invoke(q)

    results["sparse"]["answers"].append(s_response["response"])
    results["sparse"]["contexts"].append(s_response["context"])
    results["dense"]["answers"].append(d_response["response"])
    results["dense"]["contexts"].append(d_response["context"])

from datasets import Dataset

sparse_response_dataset = Dataset.from_dict({
    "question" : test_questions,
    "answer" : results["sparse"]["answers"],
    "contexts" : [[doc.page_content for doc in context_list] for context_list in results["sparse"]["contexts"]],
    "ground_truth" : test_ground_truths
})
dense_response_dataset = Dataset.from_dict({
    "question" : test_questions,
    "answer" : results["dense"]["answers"],
    "contexts" : [[doc.page_content for doc in context_list] for context_list in results["dense"]["contexts"]],
    "ground_truth" : test_ground_truths
})

In [39]:
# Load ragas evaluation metrics. We would use all possible metrics, including:
# - Faithfulness
# - Answer relevancy
# - Answer correctness
# - retrieved context related metrics
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

metrics = [
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
]

from ragas.run_config import RunConfig
run_cfg = RunConfig(
    timeout      = 120,
    max_workers  = 2,
    max_retries  = 6,
)

In [40]:
# Use ragas evaluator to report the score of each pipeline, using the metrics defined above.
from ragas import evaluate

sparse_scores = evaluate(
    sparse_response_dataset,
    metrics = metrics,
    llm = ragas_llm,
    embeddings = ragas_embedding,
    run_config = run_cfg,
    raise_exceptions=False,
)

dense_scores = evaluate(
    dense_response_dataset,
    metrics = metrics,
    llm = ragas_llm,
    embeddings = ragas_embedding,
    run_config = run_cfg,
    raise_exceptions=False,
)

print(f"Sparse RAG Score: {sparse_scores}")
print(f"Dense RAG Score: {dense_scores}")

Evaluating:   0%|          | 0/45 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/45 [00:00<?, ?it/s]

Sparse RAG Score: {'faithfulness': 0.5551, 'answer_relevancy': 0.1111, 'answer_correctness': 0.3328, 'context_recall': 0.1944, 'context_precision': 0.2130}
Dense RAG Score: {'faithfulness': 0.4401, 'answer_relevancy': 0.1111, 'answer_correctness': 0.4824, 'context_recall': 0.2778, 'context_precision': 0.1667}


In [41]:
sparse_scores.to_pandas().head(3)

,user_input,retrieved_contexts,response,reference,faithfulness,answer_relevancy,answer_correctness,context_recall,context_precision
0,What are the benifits of using pomegranite in ...,[golden. Remove pan from heat. Working very qu...,System: You are a helpful assistant. Answer th...,Pomegranate adds a unique flavor and texture t...,1.0,0.0,0.219278,0.0,0.0
1,How is A.1. Original Sauce utilized in the rec...,[Ingredients: 1 lb. boneless beef sirloin stea...,System: You are a helpful assistant. Answer th...,A.1. Original Sauce is used in the recipe by t...,1.0,1.0,0.662167,1.0,1.0
2,Wut can I find at Walmart for my baking?,[Ingredients: 2 Asian pears 2 oranges 1 banana...,System: You are a helpful assistant. Answer th...,You can find all purpose Greek seasoning at Wa...,0.8,0.0,0.211365,0.0,0.0


In [42]:
dense_scores.to_pandas().head(3)

,user_input,retrieved_contexts,response,reference,faithfulness,answer_relevancy,answer_correctness,context_recall,context_precision
0,What are the benifits of using pomegranite in ...,[Ingredients: 3 pomegranates (we're going to u...,System: You are a helpful assistant. Answer th...,Pomegranate adds a unique flavor and texture t...,0.571429,0.0,0.549098,0.5,0.0
1,How is A.1. Original Sauce utilized in the rec...,[Ingredients: 1 lb. boneless beef sirloin stea...,System: You are a helpful assistant. Answer th...,A.1. Original Sauce is used in the recipe by t...,0.500000,0.0,0.736429,1.0,1.0
2,Wut can I find at Walmart for my baking?,"[Ingredients: 1 cup unsalted butter, room temp...",System: You are a helpful assistant. Answer th...,You can find all purpose Greek seasoning at Wa...,0.058824,0.0,0.204977,0.0,0.0


In [43]:
import pandas as pd

# These are EvaluationResult objects
df_sparse = sparse_scores.to_pandas()
df_dense = dense_scores.to_pandas()

# Average the scores across samples for each metric
df_sparse_mean = df_sparse.mean(numeric_only=True).reset_index()
df_sparse_mean.columns = ['Metric', 'Sparse Retriever']

df_dense_mean = df_dense.mean(numeric_only=True).reset_index()
df_dense_mean.columns = ['Metric', 'Dense Retriever']

# Merge and compute delta
df_merged = pd.merge(df_sparse_mean, df_dense_mean, on='Metric')
df_merged['Delta'] = df_merged['Dense Retriever'] - df_merged['Sparse Retriever']

df_merged

,Metric,Sparse Retriever,Dense Retriever,Delta
0,faithfulness,0.555062,0.440059,-1.150032e-01
1,answer_relevancy,0.111111,0.111111,-3.885781e-16
2,answer_correctness,0.332780,0.482389,1.496091e-01
3,context_recall,0.194444,0.277778,8.333333e-02
4,context_precision,0.212963,0.166667,-4.629630e-02


#### Report 3: (1 point)

Compare the automated evaluation (using ragas) with your manual evaluation from the previous step. In your report, make sure to address the following:
1. How are the two evaluation methods different? Briefly describe what makes the automated evaluation distinct from your manual judgment process (e.g., consistency, objectivity, criteria used).
2.	Do both evaluations show the same results? Were the rankings or judgments about the quality of responses consistent between your analysis and the automated scores?
3.	If there were differences, why might that be? Reflect on what factors could lead to different results.

[![A Comparison of Manual Testing vs. Automation Testing - Lotus QA ...](https://tse2.mm.bing.net/th?id=OIP.uWnvDsblQboejz8ZplZnLgAAAA\&pid=Api)](https://www.lotus-qa.com/blog/manual-testing-vs-automation-testing-comparison/)


### 1. Differences Between Automated and Manual Evaluation

**Automated Evaluation with RAGAS:**

* **Consistency and Objectivity:** RAGAS employs predefined metrics and algorithms, ensuring uniform application across evaluations, thereby minimizing human bias.
* **Scalability:** Capable of processing large datasets efficiently, making it suitable for extensive evaluations.
* **Speed:** Automated processes deliver rapid results, facilitating quicker iterations in development cycles.
* **Metric-Based Assessment:** Focuses on quantifiable aspects such as context relevance and faithfulness, providing measurable insights into performance.

**Manual Evaluation:**

* **Human Insight:** Evaluators can assess nuances, context, and subtleties that automated systems might overlook.
* **Flexibility:** Humans can adapt evaluation criteria dynamically based on context and specific requirements.
* **Subjectivity:** Prone to individual biases and inconsistencies, which can affect the reliability of results.
* **Resource Intensive:** Time-consuming and less scalable, especially for large datasets.



### 2. Consistency Between Evaluations

While both evaluation methods aim to assess the quality and relevance of responses, discrepancies can arise due to their inherent differences. Automated evaluations provide consistent and objective metrics but may miss contextual subtleties. Manual evaluations, conversely, can capture nuanced insights but may lack consistency across different evaluators.



### 3. Reasons for Discrepancies

* **Contextual Understanding:** Automated systems may not fully grasp the context or intent behind certain responses, leading to potential misjudgments.
* **Subjectivity in Manual Evaluation:** Individual biases and interpretations can lead to variability in manual assessments.
* **Metric Limitations:** Automated metrics may not encompass all dimensions of quality, such as creativity or emotional resonance, which humans can perceive.
* **Data Representation:** The quality and representativeness of data used in automated evaluations can influence outcomes, potentially leading to discrepancies with human judgments.


#### Question 8: (2 points)

1. Explain the underlying mechanism and techniques used by `ragas` to evaluate the performance of RAG pipelines. How does it ensure that the evaluation is both comprehensive and relevant to the task?

RAGAS (Retrieval-Augmented Generation Assessment Suite) is a specialized evaluation framework designed to assess the performance of Retrieval-Augmented Generation (RAG) pipelines. Unlike traditional evaluation methods that often rely on surface-level metrics or manual assessments, RAGAS offers a comprehensive, automated approach that scrutinizes both the retrieval and generation components of RAG systems.

RAGAS employs a suite of metrics to provide a multifaceted evaluation:

1. **Faithfulness**: Assesses whether the generated answer accurately reflects the information present in the retrieved context. This metric ensures that the model's responses are grounded in the provided evidence, thereby minimizing hallucinations.

2. **Answer Relevancy**: Evaluates the pertinence of the generated answer to the original query, ensuring that responses are not only accurate but also directly address the user's question.&#x20;

3. **Context Precision and Recall**: These metrics measure the quality of the retrieved documents. Context Precision assesses the proportion of retrieved documents that are relevant, while Context Recall evaluates the extent to which all relevant documents have been retrieved. Together, they provide insight into the effectiveness of the retrieval component.

4. **Answer Correctness**: Compares the generated answer to a ground truth reference, if available, to determine the accuracy of the response. This metric is particularly useful when benchmark datasets with known answers are used.&#x20;

RAGAS integrates both computational and model-based techniques to ensure a thorough evaluation:

* **Model-Based Evaluation**: Utilizes large language models (LLMs) to assess aspects like faithfulness and answer relevancy. By leveraging the capabilities of LLMs, RAGAS can perform nuanced evaluations that consider the semantics and context of the content.

* **Automated Test Set Generation**: RAGAS can generate synthetic test sets by creating queries and corresponding ground truths from existing documents. This feature facilitates continuous evaluation without the need for extensive manual annotation.
* **Integration with Existing Pipelines**: Designed to work seamlessly with frameworks like LangChain and Haystack, RAGAS can be incorporated into existing RAG pipelines, enabling real-time evaluation and monitoring.

### Advantages Over Traditional Evaluation Methods

* **Holistic Assessment**: By evaluating both retrieval and generation components, RAGAS provides a more complete picture of a RAG system's performance.

* **Automated and Scalable**: The framework's automation capabilities allow for scalable evaluations, making it suitable for large-scale applications and continuous integration workflows.

* **Task Relevance**: The metrics employed by RAGAS are specifically designed to address the unique challenges of RAG systems, ensuring that evaluations are aligned with the objectives of such pipelines.



### 3.6 (Optional) Other strategy: (3 points)

#### Question 9: (3 points)

There are other retriever strategies you can use to improve the performance of your RAG pipeline. In this task:
1. Explain what the `MultiQueryRetriever` does and how it can help improve retrieval quality in your pipeline.
2. Implement the `MultiQueryRetriever` in your RAG pipeline and evaluate its performance using both manual and automated methods.
3. You may also make additional improvements to your pipeline. If you do so, briefly explain what changes you made, how they affect the system, and why they might improve performance.

In [ ]:
results["multiquery"] = {
    "answers": [],
    "contexts": []
}

for question in test_questions:
    q = {"question": question}

    mq_answer = multiquery_rag.invoke(q)
    mq_contexts = multiquery_retriever.invoke(question)

    results["multiquery"]["answers"].append(mq_answer)
    results["multiquery"]["contexts"].append([doc.page_content for doc in mq_contexts])


In [ ]:
multiquery_response_dataset = Dataset.from_dict({
    "question": test_questions,
    "answer": results["multiquery"]["answers"],
    "contexts": results["multiquery"]["contexts"],
    "ground_truth": test_ground_truths
})


In [ ]:
from ragas import evaluate

multiquery_scores = evaluate(multiquery_response_dataset,    metrics=metrics,
    llm=llm,
    embeddings=embedding_model,
    run_config=RunConfig(max_workers=1, timeout=60))

In [ ]:
print(f"Multiquery : {multiquery_scores}")

In [ ]:
multiquery_scores.to_pandas().head(3)

### 1. Understanding the MultiQueryRetriever

The `MultiQueryRetriever` is an advanced retrieval strategy within the LangChain framework designed to enhance the performance of Retrieval-Augmented Generation (RAG) pipelines. Instead of relying on a single user query, it leverages a language model (LLM) to generate multiple semantically diverse reformulations of the original question. Each of these reformulated queries is then used to retrieve documents from the vector store, and the union of these results is compiled to provide a richer and more comprehensive context for the generation phase.

**Key Benefits:**

* **Enhanced Recall:** By exploring various phrasings of the user's question, the retriever can uncover relevant documents that might be missed with a single query formulation.

* **Robustness to Query Variability:** It mitigates the sensitivity of retrieval systems to the specific wording of queries, ensuring more consistent performance across diverse inputs.

* **Improved Contextual Diversity:** Aggregating results from multiple queries introduces a broader range of perspectives, which can lead to more informative and accurate responses.


### 2. Integrating and Evaluating the MultiQueryRetriever in Your RAG Pipeline

**Integration Steps:**

1. **Replace Existing Retriever:**
   Substitute your current retriever (e.g., TF-IDF or semantic retriever) with the `MultiQueryRetriever` as shown above.

2. **Update Retrieval Logic:**
   Ensure that your pipeline uses the `multi_query_retriever` for fetching documents based on user queries.

3. **Maintain Consistent Prompting:**
   Continue using your existing prompt templates and LLM configurations for the generation phase, as the retrieval enhancement does not necessitate changes downstream.

**Evaluation Approach:**

* **Automated Evaluation with RAGAS:**
  Utilize the RAGAS framework to assess the performance of your updated pipeline. RAGAS provides metrics such as Faithfulness, Answer Relevance, and Context Precision, offering a quantitative measure of improvements.

* **Manual Evaluation:**
  Conduct a qualitative assessment by comparing responses from the original and updated pipelines. Focus on aspects like the accuracy of information, completeness of answers, and the relevance of retrieved context.

**Expected Outcomes:**

Integrating the `MultiQueryRetriever` is anticipated to enhance the retrieval phase, leading to more informative and accurate responses. This improvement should be reflected in both automated metrics and manual assessments, demonstrating the effectiveness of the multi-query approach.

### 3. Additional Enhancements to the RAG Pipeline

Beyond incorporating the `MultiQueryRetriever`, consider the following enhancements to further improve your RAG pipeline's performance:

* **Custom Query Generation Prompts:**
  Tailor the prompt used by the LLM to generate query variations, aligning it more closely with your domain-specific language and user intents.

* **Incorporate Reranking Mechanisms:**
  Implement a reranking step using models like BERT or other transformer-based architectures to prioritize the most relevant documents from the retrieved set.

* **Metadata Filtering:**
  Leverage document metadata (e.g., publication date, author, source) to filter and prioritize retrieved documents, enhancing the relevance of the context provided to the LLM.

* **Dynamic Query Expansion:**
  Incorporate user feedback or interaction history to dynamically adjust and expand queries, ensuring the retrieval process adapts to the user's evolving information needs.

**Impact of Enhancements:**

These additional improvements aim to refine both the retrieval and generation phases of your RAG pipeline. By customizing query generation, reranking results, and utilizing metadata, you can provide more precise and contextually appropriate information to the LLM, leading to higher-quality responses and improved user satisfaction.



## 4. Read more: (10 points)

#### Cache-Augmented Generation (CAG): (4 points)

1. What is Cache-Augmented Generation (CAG)? How does it improve efficiency or performance during generation?
2. What are the similarities and differences between Cache-Augmented Generation (CAG) and Retrieval-Augmented Generation (RAG)? In what scenarios might you prefer one over the other?

### 1. What is Cache-Augmented Generation (CAG)?

Cache-Augmented Generation (CAG) is a technique designed to improve the efficiency and responsiveness of large language models (LLMs) by preloading relevant information into the model's context window and caching its internal state. The core idea is to eliminate the need for real-time document retrieval during inference by front-loading the knowledge the model will need and reusing it across multiple queries.

The process involves selecting and injecting relevant knowledge (such as documents, facts, or structured data) into the input of the LLM. Once the model processes this information, its internal key-value (KV) cache—used by transformer architectures to track attention and context—is stored. During inference, new queries are run in the context of this preloaded and precomputed state.

This design improves performance by removing the computational cost and latency of retrieval at inference time. It also simplifies system architecture, as there's no need to manage retrieval systems or vector databases at runtime.

However, CAG has limitations. It is best suited for static or rarely updated knowledge bases, as any change in the content requires regenerating the cache. Additionally, the amount of context that can be cached is constrained by the LLM's context window size, making it essential to carefully curate the information being included.


### 2. How CAG Compares to Retrieval-Augmented Generation (RAG)

Both CAG and RAG aim to enhance language model outputs by incorporating external knowledge that isn't encoded directly in the model's weights. They both solve the problem of limited model memory and out-of-date training data. However, their methods and applications differ significantly.

CAG preloads and caches knowledge ahead of time. This means all relevant context is already available during inference, which allows for faster responses and avoids repeated retrieval operations. Because there's no external lookup step during query time, the system is more efficient and easier to deploy in latency-sensitive environments.

RAG, in contrast, performs retrieval dynamically at inference time. When a query is submitted, it is used to fetch relevant documents from an external source—typically a vector database using embeddings. These retrieved documents are then fed to the LLM along with the query, allowing it to generate a context-aware response based on current information.

This makes RAG highly adaptable and suitable for applications where the knowledge base is large, diverse, or frequently updated. However, it also introduces greater complexity, higher latency, and greater infrastructure requirements.

In terms of use cases, CAG is preferable when the knowledge base is static and the system requires low-latency responses—such as with technical manuals, customer support chatbots with stable content, or internal documentation. RAG is more appropriate when the system must respond with the most current or diverse information—such as in news summarization, research assistants, or systems handling large enterprise knowledge graphs.


#### Multi-modal RAG: (6 points)

1. How do models like CLIP enable the embedding of both text and images into a shared vector space? What are the advantages and disadvantages of using a unified embedding space for cross-modal retrieval in RAG systems.
2. In systems like [Colpali](https://arxiv.org/pdf/2407.01449), how does dividing document images into patches enhance the retrieval process in multimodal RAG? Explore how patch-based processing preserves structural information and its impact on retrieval accuracy.
3. What are the implications of converting non-text modalities (e.g., images) into textual representations for retrieval purposes? Discuss the benefits and drawbacks of grounding all modalities into a primary modality, such as text, in the context of RAG.

`# WRITE YOUR ANSWER HERE`